[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/georgianpartners/Multimodal-Toolkit/blob/master/notebooks/text_w_tabular_classification.ipynb)

# Training a BertWithTabular Model for Clothing Review Recommendation Prediction

This guide follows closely with the [example](https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/trainer/01_text_classification.ipynb#scrollTo=bwl3I_VGAZXb) from HuggingFace for text classificaion on the GLUE dataset.

Install `multimodal-transformers`, `kaggle`  so we can get the dataset.

## All other imports are here:

In [1]:
from dataclasses import dataclass, field
import json
import logging
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
from typing import Optional

import numpy as np
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoConfig,
    Trainer,
    EvalPrediction,
    set_seed
)
from transformers.training_args import TrainingArguments

from multimodal_transformers.data import load_data_from_folder,load_data_into_folds
from multimodal_transformers.model import TabularConfig
from multimodal_transformers.model import AutoModelWithTabular

logging.basicConfig(level=logging.INFO)
os.environ['COMET_MODE'] = 'DISABLED'
import csv

/data/chenxi/anaconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data/chenxi/anaconda3/envs/myenv/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /data/chenxi/anaconda3/envs/myenv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/data/chenxi/anaconda3/envs/myenv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: /data/chenxi/anaconda3/envs/myenv did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/data/chenxi/anaconda3/envs/myenv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/data/chenxi/anaconda3/envs/myenv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/data/chenxi/anaconda3/envs/myenv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcuda

#### Let us take a look at what the dataset looks like

In [2]:
from sklearn.preprocessing import LabelEncoder
from io import StringIO
file_path='/data/chenxi/llm-feature-engeneering/dataset/diabetes.arff'
with open(file_path, 'r') as f:
    lines = f.readlines()

# Extract attribute names
attributes = []
for line in lines:
    if line.startswith("@attribute"):
        attributes.append(line.split()[1])

# Extract data
data_start_index = lines.index("@data\n") + 1
data_lines = "\n".join(lines[data_start_index:])

# Convert data lines to DataFrame
df = pd.read_csv(StringIO(data_lines), header=None, names=attributes, na_values="?")

# Replace missing values with -1
df.fillna(-1, inplace=True)

# Convert categorical string data into numbers
for column in df.columns:
    if df[column].dtype == 'object':  # Check if the column is of object type (string)
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])


In [3]:
df.columns.tolist()

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',
 'Outcome']

In [4]:
data_df=df.copy()

In [5]:
column22 = pd.read_csv('/data/chenxi/llm-feature-engeneering/src/model/responses/diabetes/analysis.csv')
column22 = column22.reset_index(drop=True)
data_df = data_df.reset_index(drop=True)

data_df['response'] = column22['analysis']

In [6]:
data_df.to_csv('/data/chenxi/llm-feature-engeneering/src/Fine_tune/diabetes/dataset/clean.csv')

We see that the data contains both text in the `Review Text` and `Title` column as well as tabular features in the `Division Name`, `Department Name`, and `Class Name` columns.

In [7]:
data_df.describe(include=np.object)

/tmp/ipykernel_1429392/499234424.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data_df.describe(include=np.object)


,response
count,768
unique,768
top,"Based on the patient profile provided, there i..."
freq,1


In [8]:
data_df.head(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,response
0,6,148,72,35,0,33.6,0.627,50,1,"Based on the patient profile provided, there i..."
1,1,85,66,29,0,26.6,0.351,31,0,"Additionally, the presence of a murmur, its mo..."
2,8,183,64,0,0,23.3,0.672,32,1,Based on the presence of a murmur and the most...
3,1,89,66,23,94,28.1,0.167,21,0,"Based on the given patient profile, there is a..."
4,0,137,40,35,168,43.1,2.288,33,1,"Based on the information provided, the presenc..."


In [9]:
# data_df['Outcome'] = data_df['Outcome'].apply(lambda x: np.random.randint(2) if x in [0, 1] else x)
# data_df.head(5)

In this demonstration, we split our data into 8:1:1 training splits. We also save our splits to `train.csv`, `val.csv`, and `test.csv` as this is the format our dataloader requires.


In [10]:
# train_df, val_df, test_df = np.split(data_df.sample(frac=1), [int(.8*len(data_df)), int(.9 * len(data_df))])
# print('Num examples train-val-test')
# print(len(train_df), len(val_df), len(test_df))
# train_df.to_csv('/data/chenxi/llm-feature-engeneering/src/Fine_tune/diabetes/dataset/train.csv')
# val_df.to_csv('//data/chenxi/llm-feature-engeneering/src/Fine_tune/diabetes/dataset/val.csv')
# test_df.to_csv('/data/chenxi/llm-feature-engeneering/src/Fine_tune/diabetes/dataset/test.csv')

In [11]:
# import os
# from sklearn.model_selection import train_test_split

# # Number of experiments
# num_experiments = 5

# for seed in range(num_experiments):
#     # Split data into train and test sets
#     train_df, test_df = train_test_split(data_df, test_size=0.2, random_state=seed)
    
#     # Further split train set into train and validation sets
#     train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=seed)
    
#     # Create the directory for the current seed if it doesn't exist
#     seed_dir = f'/data/chenxi/llm-feature-engeneering/src/Fine_tune/diabetes/data_seed_{seed}/'
#     if not os.path.exists(seed_dir):
#         os.makedirs(seed_dir)
    
#     # Save the splits
#     test_df.to_csv(f'{seed_dir}test.csv')
#     train_df.to_csv(f'{seed_dir}train.csv')
#     val_df.to_csv(f'{seed_dir}val.csv')

## We then our Experiment Parameters
We use Data Classes to hold each of our arguments for the model, data, and training.

In [12]:
@dataclass
class ModelArguments:
  """
  Arguments pertaining to which model/config/tokenizer we are going to Fine_tune from.
  """

  model_name_or_path: str = field(
      metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
  )
  config_name: Optional[str] = field(
      default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
  )
  tokenizer_name: Optional[str] = field(
      default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
  )
  cache_dir: Optional[str] = field(
      default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
  )


@dataclass
class MultimodalDataTrainingArguments:
  """
  Arguments pertaining to how we combine tabular features
  Using `HfArgumentParser` we can turn this class
  into argparse arguments to be able to specify them on
  the command line.
  """

  data_path: str = field(metadata={
                            'help': 'the path to the csv file containing the dataset'
                        })
  column_info_path: str = field(
      default=None,
      metadata={
          'help': 'the path to the json file detailing which columns are text, categorical, numerical, and the label'
  })

  column_info: dict = field(
      default=None,
      metadata={
          'help': 'a dict referencing the text, categorical, numerical, and label columns'
                  'its keys are text_cols, num_cols, cat_cols, and label_col'
  })

  categorical_encode_type: str = field(default='ohe',
                                        metadata={
                                            'help': 'sklearn encoder to use for categorical data',
                                            'choices': ['ohe', 'binary', 'label', 'none']
                                        })
  numerical_transformer_method: str = field(default='yeo_johnson',
                                            metadata={
                                                'help': 'sklearn numerical transformer to preprocess numerical data',
                                                'choices': ['yeo_johnson', 'box_cox', 'quantile_normal', 'none']
                                            })
  task: str = field(default="classification",
                    metadata={
                        "help": "The downstream training task",
                        "choices": ["classification", "regression"]
                    })

  mlp_division: int = field(default=4,
                            metadata={
                                'help': 'the ratio of the number of '
                                        'hidden dims in a current layer to the next MLP layer'
                            })
  combine_feat_method: str = field(default='individual_mlps_on_cat_and_numerical_feats_then_concat',
                                    metadata={
                                        'help': 'method to combine categorical and numerical features, '
                                                'see README for all the method'
                                    })
  mlp_dropout: float = field(default=0.1,
                              metadata={
                                'help': 'dropout ratio used for MLP layers'
                              })
  numerical_bn: bool = field(default=True,
                              metadata={
                                  'help': 'whether to use batchnorm on numerical features'
                              })
  use_simple_classifier: str = field(default=True,
                                      metadata={
                                          'help': 'whether to use single layer or MLP as final classifier'
                                      })
  mlp_act: str = field(default='relu',
                        metadata={
                            'help': 'the activation function to use for finetuning layers',
                            'choices': ['relu', 'prelu', 'sigmoid', 'tanh', 'linear']
                        })
  gating_beta: float = field(default=0.2,
                              metadata={
                                  'help': "the beta hyperparameters used for gating tabular data "
                                          "see https://www.aclweb.org/anthology/2020.acl-main.214.pdf"
                              })

  def __post_init__(self):
      assert self.column_info != self.column_info_path
      if self.column_info is None and self.column_info_path:
          with open(self.column_info_path, 'r') as f:
              self.column_info = json.load(f)

### Here are the data and training parameters we will use.
For model we can specify any supported HuggingFace model classes (see README for more details) as well as any AutoModel that are from the supported model classes. For the data specifications, we need to specify a dictionary that specifies which columns are the `text` columns, `numerical feature` columns, `categorical feature` column, and the `label` column. If we are doing classification, we can also specify what each of the labels means in the label column through the `label list`. We can also specifiy these columns using a path to a json file with the argument `column_info_path` to `MultimodalDataTrainingArguments`.

In [13]:
text_cols = ['response']
cat_cols = ['Age']

numerical_cols = ['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction'
 ]
# numerical_cols = ['num']
column_info_dict = {
    'text_cols': text_cols,
    'num_cols': numerical_cols,
    'cat_cols': cat_cols,
    'label_col': 'Outcome',
    'label_list': ["The patient has diebetes", "The patient does not have diebetes"]
}


model_args = ModelArguments(
    model_name_or_path='distilbert-base-uncased'
)

data_args = MultimodalDataTrainingArguments(
    # data_path='/data/chenxi/3/3/clean.csv',
    data_path='/data/chenxi/llm-feature-engeneering/src/Fine_tune/diabetes/data_seed_4',
    combine_feat_method='gating_on_cat_and_num_feats_then_sum',
    column_info=column_info_dict,
    task='classification'
)

training_args = TrainingArguments(
    output_dir="/data/chenxi/llm-feature-engeneering/src/Fine_tune/diabetes/model/5",
    logging_dir="/data/chenxi/llm-feature-engeneering/src/Fine_tune/diabetes//runs",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    num_train_epochs=60,
    per_device_train_batch_size=40,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps=20,
    evaluation_strategy="epoch",
    save_strategy="epoch",  # Change this to match the evaluation_strategy
    logging_first_step=True,
    learning_rate=2e-5,
    adafactor=True,
    gradient_accumulation_steps=1,
    lr_scheduler_type="cosine",
    load_best_model_at_end=True,
    seed=42,
    fp16=True,
)

set_seed(training_args.seed)

/data/chenxi/anaconda3/envs/myenv/lib/python3.10/site-packages/transformers/training_args.py:1356: FutureWarning: `--adafactor` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--optim adafactor` instead
  warnings.warn(


## Now we can load our model and data.
### We first instantiate our HuggingFace tokenizer
This is needed to prepare our custom torch dataset. See `torch_dataset.py` for details.

In [14]:
tokenizer_path_or_name = model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path
print('Specified tokenizer: ', tokenizer_path_or_name)
tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_path_or_name,
    cache_dir=model_args.cache_dir,
)

Specified tokenizer:  distilbert-base-uncased


### Load dataset csvs to torch datasets
The function `load_data_from_folder` expects a path to a folder that contains `train.csv`, `test.csv`, and/or `val.csv` containing the respective split datasets.

In [15]:
# Get Datasets
train_dataset, val_dataset, test_dataset = load_data_from_folder(
    data_args.data_path,
    data_args.column_info['text_cols'],
    tokenizer,
    label_col=data_args.column_info['label_col'],
    label_list=data_args.column_info['label_list'],
    categorical_cols=data_args.column_info['cat_cols'],
    numerical_cols=data_args.column_info['num_cols'],
    sep_text_token_str=tokenizer.sep_token,
)

INFO:multimodal_transformers.data.data_utils:7 numerical columns
/data/chenxi/anaconda3/envs/myenv/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (491). n_quantiles is set to n_samples.
  warnings.warn(
INFO:multimodal_transformers.data.data_utils:52 categorical columns
INFO:multimodal_transformers.data.data_utils:7 numerical columns
INFO:multimodal_transformers.data.load_data:Text columns: ['response']
INFO:multimodal_transformers.data.load_data:Raw text example: Based on the patient profile provided, there is no mention of the patient's sex. However, assuming this patient is female based on the pregnancy status, it is important to consider the potential impact of hormonal changes during pregnancy on glucose levels. The elevated glucose level of '148.0' mg/dL could suggest gestational diabetes or impaired glucose tolerance, which is relatively common during pregnancy.

The blood pressure rea

INFO:multimodal_transformers.data.load_data:Raw text example: Based on the given data, there is no information about the presence or absence of a murmur, its location, or its characteristics. However, if a murmur is present, other factors such as age, sex, and the most audible location of the murmur could be examined to determine possible correlations. For example, certain types of murmurs are more common in specific age groups or genders. Additionally, factors like height, weight, and pregnancy status might influence the likelihood of having a murmur. It is also crucial to consider auscultation locations as they can provide valuable information about the underlying cause of the murmur. Comparing this patient's data with other patients with murmur characteristics could provide insights into potential associations. Finally, the campaign data could be utilized to determine if any interventions or treatments have been implemented and their impact on the patient's overall health.


In [16]:
num_labels = len(np.unique(train_dataset.labels))
# num_labels = 2
num_labels

2

In [17]:
config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )
tabular_config = TabularConfig(num_labels=num_labels,
                               cat_feat_dim=train_dataset.cat_feats.shape[1],
                               numerical_feat_dim=train_dataset.numerical_feats.shape[1],
                               **vars(data_args))
config.tabular_config = tabular_config

In [18]:
model = AutoModelWithTabular.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        config=config,
        cache_dir=model_args.cache_dir
    )

Some weights of DistilBertWithTabular were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'tabular_combiner.h_cat_layer.weight', 'classifier.bias', 'tabular_combiner.g_cat_layer.bias', 'tabular_combiner.g_num_layer.bias', 'tabular_combiner.h_num_layer.weight', 'tabular_combiner.g_num_layer.weight', 'tabular_classifier.bias', 'tabular_combiner.num_bn.running_var', 'tabular_combiner.layer_norm.bias', 'tabular_classifier.weight', 'tabular_combiner.h_bias', 'tabular_combiner.num_bn.bias', 'tabular_combiner.num_bn.weight', 'tabular_combiner.layer_norm.weight', 'tabular_combiner.num_bn.running_mean', 'pre_classifier.bias', 'tabular_combiner.num_bn.num_batches_tracked', 'pre_classifier.weight', 'tabular_combiner.g_cat_layer.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### We need to define a task-specific way of computing relevant metrics:

In [19]:
import numpy as np
from scipy.special import softmax
from sklearn.metrics import (
    auc,
    precision_recall_curve,
    roc_auc_score,
    f1_score,
    confusion_matrix,
    matthews_corrcoef,
)

def calc_classification_metrics(p: EvalPrediction):
    predictions = p.predictions[0]
    pred_labels = np.argmax(predictions, axis=1)
    pred_scores = softmax(predictions, axis=1)[:, 1]
    labels = p.label_ids
    acc = (pred_labels == labels).mean() 
    if len(np.unique(labels)) == 2:  # binary classification
        roc_auc_pred_score = roc_auc_score(labels, pred_scores)
        precisions, recalls, thresholds = precision_recall_curve(labels,
                                                                    pred_scores)
        fscore = (2 * precisions * recalls) / (precisions + recalls)
        fscore[np.isnan(fscore)] = 0
        ix = np.argmax(fscore)
        threshold = thresholds[ix].item()
        pr_auc = auc(recalls, precisions)
        tn, fp, fn, tp = confusion_matrix(labels, pred_labels, labels=[0, 1]).ravel()
        result = {'roc_auc': roc_auc_pred_score,
                    'threshold': threshold,
                    'pr_auc': pr_auc,
                    'recall': recalls[ix].item(),
                    'precision': precisions[ix].item(), 'f1': fscore[ix].item(),
                    'tn': tn.item(), 'fp': fp.item(), 'fn': fn.item(), 'tp': tp.item(),
                    'acc': acc,
                    }
    else:
        acc = (pred_labels == labels).mean()
        f1 = f1_score(y_true=labels, y_pred=pred_labels)
        result = {
            "acc": acc,
            "f1": f1,
            "acc_and_f1": (acc + f1) / 2,
            "mcc": matthews_corrcoef(labels, pred_labels)
        }

    return result

In [20]:
trainer = Trainer(
    model=model.to(0),
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=calc_classification_metrics,
)

## Launching the training is as simple is doing trainer.train() 🤗

In [21]:
%%time
trainer.train()

Epoch,Training Loss,Validation Loss,Roc Auc,Threshold,Pr Auc,Recall,Precision,F1,Tn,Fp,Fn,Tp,Acc
1,0.682100,0.672034,0.467340,0.463976,0.251264,1.000000,0.270492,0.425806,90,0,33,0,0.731707
2,0.681600,0.659581,0.464478,0.453950,0.251856,0.939394,0.281818,0.433566,90,0,33,0,0.731707
3,0.681600,0.643241,0.476431,0.429938,0.268121,0.939394,0.279279,0.430556,90,0,33,0,0.731707
4,0.672600,0.624885,0.480808,0.396010,0.286510,1.000000,0.275000,0.431373,90,0,33,0,0.731707
5,0.658500,0.611521,0.485017,0.371393,0.269468,0.969697,0.278261,0.432432,90,0,33,0,0.731707
6,0.658500,0.602493,0.483670,0.350032,0.267625,0.969697,0.280702,0.435374,90,0,33,0,0.731707
7,0.653300,0.601021,0.490067,0.345933,0.261183,0.969697,0.280702,0.435374,90,0,33,0,0.731707
8,0.647100,0.605919,0.477609,0.349533,0.253339,1.000000,0.272727,0.428571,90,0,33,0,0.731707
9,0.647100,0.609843,0.458418,0.362941,0.268239,0.909091,0.280374,0.428571,90,0,33,0,0.731707
10,0.653900,0.596823,0.447643,0.324342,0.261778,0.878788,0.292929,0.439394,90,0,33,0,0.731707


/tmp/ipykernel_1429392/3478532714.py:22: RuntimeWarning: invalid value encountered in true_divide
  fscore = (2 * precisions * recalls) / (precisions + recalls)
/tmp/ipykernel_1429392/3478532714.py:22: RuntimeWarning: invalid value encountered in true_divide
  fscore = (2 * precisions * recalls) / (precisions + recalls)
/tmp/ipykernel_1429392/3478532714.py:22: RuntimeWarning: invalid value encountered in true_divide
  fscore = (2 * precisions * recalls) / (precisions + recalls)
/tmp/ipykernel_1429392/3478532714.py:22: RuntimeWarning: invalid value encountered in true_divide
  fscore = (2 * precisions * recalls) / (precisions + recalls)
/tmp/ipykernel_1429392/3478532714.py:22: RuntimeWarning: invalid value encountered in true_divide
  fscore = (2 * precisions * recalls) / (precisions + recalls)
/tmp/ipykernel_1429392/3478532714.py:22: RuntimeWarning: invalid value encountered in true_divide
  fscore = (2 * precisions * recalls) / (precisions + recalls)
/tmp/ipykernel_1429392/3478532714.

CPU times: user 3min 11s, sys: 14.3 s, total: 3min 26s
Wall time: 3min 26s


TrainOutput(global_step=780, training_loss=0.2820341967774603, metrics={'train_runtime': 206.2155, 'train_samples_per_second': 142.86, 'train_steps_per_second': 3.782, 'total_flos': 3790360032474960.0, 'train_loss': 0.2820341967774603, 'epoch': 60.0})

In [22]:
eval_results = trainer.evaluate(test_dataset)
print(eval_results)

{'eval_loss': 0.6591622233390808, 'eval_roc_auc': 0.4185491276400367, 'eval_threshold': 0.30192774534225464, 'eval_pr_auc': 0.3191105598413285, 'eval_recall': 1.0, 'eval_precision': 0.3618421052631579, 'eval_f1': 0.5314009661835749, 'eval_tn': 99, 'eval_fp': 0, 'eval_fn': 55, 'eval_tp': 0, 'eval_acc': 0.6428571428571429, 'eval_runtime': 0.3215, 'eval_samples_per_second': 478.975, 'eval_steps_per_second': 62.205, 'epoch': 60.0}


/tmp/ipykernel_1429392/3478532714.py:22: RuntimeWarning: invalid value encountered in true_divide
  fscore = (2 * precisions * recalls) / (precisions + recalls)
